In [1]:
import os
import re
import folium
import pandas as pd
import numpy as np
import datetime
from urllib.request import urlretrieve
from tqdm import tqdm_notebook as tqdm
from mpl_toolkits.basemap import Basemap

## Analyze Hurricane Paths

### 1. Load Cleaned Hurdat Data

In [4]:
# Load hurdat data formatted into dataframe
hurricanes_df = pd.read_csv("../data/hurricanes_df_cleaned.csv")
hurricanes_df[hurricanes_df['id']=='AL112017'] # IRMA in 2017

,id,name,date,time,dt,storm_status,lat,lon,max_wind,min_pressure,over_land,landfall,duration,is_hurricane,year
49896,AL112017,IRMA,20170830,0,2017-08-30 00:00:00,TD,16.1,-26.9,30.0,1008.0,1.0,1,14 days 12:00:00.000000000,0,2017
49897,AL112017,IRMA,20170830,600,2017-08-30 06:00:00,TS,16.2,-28.3,35.0,1007.0,1.0,1,14 days 12:00:00.000000000,0,2017
49898,AL112017,IRMA,20170830,1200,2017-08-30 12:00:00,TS,16.3,-29.7,45.0,1006.0,0.0,1,14 days 12:00:00.000000000,0,2017
49899,AL112017,IRMA,20170830,1800,2017-08-30 18:00:00,TS,16.3,-30.8,50.0,1004.0,0.0,1,14 days 12:00:00.000000000,0,2017
49900,AL112017,IRMA,20170831,0,2017-08-31 00:00:00,TS,16.3,-31.7,55.0,999.0,0.0,1,14 days 12:00:00.000000000,0,2017
49901,AL112017,IRMA,20170831,600,2017-08-31 06:00:00,HU,16.4,-32.5,65.0,994.0,0.0,1,14 days 12:00:00.000000000,0,2017
49902,AL112017,IRMA,20170831,1200,2017-08-31 12:00:00,HU,16.7,-33.4,80.0,983.0,0.0,1,14 days 12:00:00.000000000,0,2017
49903,AL112017,IRMA,20170831,1800,2017-08-31 18:00:00,HU,17.1,-34.2,95.0,970.0,0.0,1,14 days 12:00:00.000000000,0,2017
49904,AL112017,IRMA,20170901,0,2017-09-01 00:00:00,HU,17.5,-35.1,100.0,967.0,0.0,1,14 days 12:00:00.000000000,0,2017
49905,AL112017,IRMA,20170901,600,2017-09-01 06:00:00,HU,17.9,-36.1,100.0,967.0,0.0,1,14 days 12:00:00.000000000,0,2017


### 2. Define functions for mapping hurricane paths

In [2]:
def convert_lat_lon(value, col):
    """
    Lat/lon is encoded with the numeric value plus E/W/N/S. We want to 
    convert this into an absolute decimal value for folium to interpret.
    """
    if col=='lon':
        amount = -float(re.sub('[EW]', '', value)) if 'W' in value else float(re.sub('[EW]', '', value))
    elif col=='lat':
        amount = -float(re.sub('[NS]', '', value)) if 'S' in value else float(re.sub('[NS]', '', value))
    
    return amount

def plot_path(df, folium_map):
    """
    Generates a tuple comprising of lat/lon for the 
    folium map to plot.
    """
    # Folium requires an array of tuples
    points = df[['lat', 'lon']]
    tuple_points = [tuple(x) for x in points.values]
    return tuple_points

def hurr_category(max_wind):
    """
    Defining storm category at a point in time.
    """
    if max_wind <= 73:
        cat = 'TS'
    if (74 <= max_wind <= 95):
        cat = 1
    if (96 <= max_wind <= 110):
        cat = 2
    if (111 <= max_wind <= 129):
        cat = 3
    if (130 <= max_wind <= 156):
        cat = 4
    if max_wind >= 157:
        cat = 5
    return cat

def plot_paths_for_year(df, folium_map):
    """
    Plots hurricane path for all hurricanes in the dataframe
    where lines are color coded for the intensity (category) at that
    point in time.
    """
    # Map colors of path based on category level
    path_colors = {1: '#ffb3b3',
                   2: '#ff8080',
                   3: '#e60000',
                   4: '#ff0000',
                   5: '#580808',
                  'TS': '#ffe6e6'}
    
    for name in df['name'].unique():
        name_df = df[df['name']==name]
        for i, dt in name_df.iterrows():
            day_df = name_df.ix[i:i+1, :]
            day_points = plot_path(day_df, folium_map)
            date = name_df.loc[i, 'date']
            time = name_df.loc[i, 'time']
            category = hurr_category(name_df.loc[i, 'max_wind'])
            maxwind = name_df.loc[i, 'max_wind']
            hurr_info = f"""
            Storm:    {name}
            Date:     {date}
            Time:     {time}
            Category: {category}
            Max Wind: {maxwind}mph
            """
            folium.PolyLine(day_points, tooltip=name, popup=hurr_info, color=path_colors[category]).add_to(folium_map)
            
    return

### 3. Compare hurricane paths in 2000-2017 and 1900-1917

In [20]:
hurr_2000s = hurricanes_df[hurricanes_df['year']>=2008]
hurr_2000s_map = folium.Map(location=[30, -60],
                            zoom_start = 4)

plot_paths_for_year(hurr_2000s, hurr_2000s_map)

# Save as html
hurr_2000s_map.save('../maps/hurricane_paths_2008-2017.html')

C:\Users\Himanshu\Anaconda3\lib\site-packages\ipykernel_launcher.py:115: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


In [21]:
hurr_1900s = hurricanes_df[(hurricanes_df['year']>=1918) & (hurricanes_df['year']<=1927)]
hurr_1900s_map = folium.Map(location=[30, -60],
                            zoom_start = 4)
plot_paths_for_year(hurr_1900s, hurr_1900s_map)

# Save as html
hurr_1900s_map.save('../maps/hurricane_paths_1918-1927.html')

C:\Users\Himanshu\Anaconda3\lib\site-packages\ipykernel_launcher.py:115: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
